In [16]:
import selenium
from selenium import webdriver
import pandas as pd
import re

In [34]:
browser=webdriver.Chrome('C:/Users/Administrator/chromedriver.exe')

In [35]:
browser.get('https://www.zomato.com/bangalore/great-food-no-bull')

In [36]:
restaurants=browser.find_elements_by_css_selector('div.col-s-8.col-l-1by3')
print(len(restaurants))

141


In [37]:
def get_reviews(var,selector,attribute):
    try:
        tag=var.find_element_by_css_selector(selector)
        tag=tag.get_attribute(attribute)
    except: 
        tag=0
    return tag

In [38]:
url_info=pd.DataFrame(columns=['Restaurant Name','Restaurant ID','URL'])

In [39]:
for restaurant in restaurants:
    
    name=get_reviews(restaurant,'div.res_title.zblack.bold','title')
    name=name.lstrip()
    
    r_id=get_reviews(restaurant,'div.relative.top-res-box.entity-ads-snippet-track','data-res-id')
    
    url=get_reviews(restaurant,'a','href')
  
    rest1={'Restaurant Name':name,'Restaurant ID':r_id,'URL':url}
    url_info=url_info.append(rest1,ignore_index=True)
url_info=url_info.head(55)
url_info.head()

,Restaurant Name,Restaurant ID,URL
0,Red Rhino,18492910,https://www.zomato.com/bangalore/red-rhino-whi...
1,Bologna,18682433,https://www.zomato.com/bangalore/bologna-indir...
2,1131 Bar + Kitchen,18580202,https://www.zomato.com/bangalore/1131-bar-kitc...
3,Roots,18270163,https://www.zomato.com/bangalore/roots-koraman...
4,Brooks and Bonds Brewery,18605417,https://www.zomato.com/bangalore/brooks-and-bo...


In [40]:
restaurant_info=pd.DataFrame(columns=['Restaurant Name','Restaurant ID','User Name','User ID','Rating'])

In [41]:
c=0
for i in range(len(url_info)):
    c=c+1
    print(c)
    browser.get(url_info['URL'].iloc[i])
    try:
        r_id = url_info['Restaurant ID'].iloc[i]
        name=browser.find_element_by_css_selector('h1.res-name.left.mb0')
        name=name.find_element_by_css_selector('a')
        name=name.get_attribute('innerHTML').lstrip()
        name=re.sub('amp;','',name)
    except:
        r_id=None
        continue
    if name in url_info['Restaurant Name'].iloc[i]:
        
        while True:
            try:
                reviews=browser.find_elements_by_css_selector('div.ui.segments.res-review-body.res-review.clearfix.js-activity-root.mbti.item-to-hide-parent.stupendousact') 
                for review in reviews:
                    try:
                        user_name=review.find_element_by_css_selector('div.header.nowrap.ui.left')
                        user_name=user_name.find_element_by_css_selector('a')
                        user_name=user_name.get_attribute('innerHTML')
                        user_name=user_name.strip()
                        user_id=review.find_element_by_css_selector('div.header.nowrap.ui.left')
                        user_id=user_id.find_element_by_css_selector('a')
                        user_id=user_id.get_attribute('data-entity_id')
                    except:
                        user_name=None
                        user_id=None

                    try:
                        rating=review.find_element_by_css_selector('div.ttupper.fs12px.left.bold.zdhl2.tooltip')
                        rating=rating.get_attribute('aria-label')
                    except:
                        rating=None
                    rating=re.sub('Rated','',rating)

                    rest={'Restaurant Name':name,'Restaurant ID':r_id,'User Name':user_name,'User ID':user_id,'Rating':rating}
                    restaurant_info=restaurant_info.append(rest,ignore_index=True)
                browser.find_element_by_css_selector('div.ui.segment.clearfix.zs-load-more.res-page-load-more').click()
            except:
                break

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55


In [42]:
restaurant_info.to_csv("Reviews.csv",index=False)
restaurant_info.shape

(9695, 5)

# Number of distinct restaurants

In [43]:
restaurant_info['Restaurant Name'].nunique()

9

## Model buliding

In [ ]:
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics.pairwise import cosine_similarity
from surprise import KNNWithZScore
from surprise import SVD, dataset,Reader
import pandas as pd
import numpy as np

In [ ]:
review=pd.read_csv('C:/Users/Administrator/Desktop\Machine learning/assignment/Reviews.csv')
review.head()

In [ ]:
scaler=StandardScaler()
ui_review_matrix=review.pivot_table(columns='Restaurant ID',index='User ID',values='Rating')
null_matrix=ui_review_matrix
scaler.fit(ui_review_matrix)
# ui_review_matrix=ui_review_matrix.apply(lambda x: ((x-x.mean)/x.std()),axis=1).fillna(0)
ui_review_matrix=pd.DataFrame(scaler.transform(ui_review_matrix),columns=ui_review_matrix.columns,index=ui_review_matrix.index)
ui_review_matrix=ui_review_matrix.fillna(0)
ui_review_matrix.head()

In [ ]:
sim_matrix=pd.DataFrame(cosine_similarity(ui_review_matrix),index=ui_review_matrix.index,
                       columns=ui_review_matrix.index)
sim_matrix.head()

In [ ]:
def recommend(userid):
    neigh=sim_matrix[userid].drop(userid).sort_values(ascending=False).head(5).index
    neigh_matrix=ui_review_matrix.loc[neigh]
    user_restaraunt=null_matrix.loc[userid]
    not_gone=user_restaraunt[pd.isnull(user_restaraunt)]
    pred=neigh_matrix.mean()[not_gone.index]
    recommend=pred.sort_values(ascending=False).head().index
    data=review[review['Restaurant ID'].isin(recommend)]['Restaurant Name'].unique()
    return pd.DataFrame(data).rename(columns={0:'Restaraunt Name'})

In [ ]:
recommend(11876)